In [35]:
import os 
import pandas as pd 
import re

datapath = './Project3CommunityData'
clean_data = 'clean_data'


ping_cols = ['website', 'date', 'time', 'latency_times', 'latency_avg']
#www.cnn.com,05-13-2020,10:00:01,12.8|16.9|8.73|8.61|7.37|7.72|8.63|7.27|15.1|8.29|8.37,9.98090909090909

tr1_cols = ['website', 'date', 'time', 'hop_idx', 'hop_ip', 'latency1', 'latency2', 'latency3', 'latency_avg']
# www.cnn.com,05-13-2020,10:00:01,1,(10.0.0.1),1.558,2.584,3.525,2.5556666666666668

tr2_cols = ['website', 'date', 'time', 'hop_count']
# www.cnn.com,05-16-2020,14:32:47,11

api_cols = ['website', 'call', 'date', 'time', 'latency']
# www.amazon.com,https://www.amazon.com/s?k=networks&ref=nb_sb_noss_2,05-13-2020,14:36:27,41.97399999999999

pchar_agg_cols = ['website', 'date', 'time', 'hops']
# en.wikipedia.org,05-13-2020,14:38:49,14

pchar1_cols = ['website', 'date', 'time', 'hop_idx', 'dst', '?' ,'??']
# en.wikipedia.org,05-13-2020,14:38:49,1,10.0.2.2,0.000000,0

pchar2_cols = []
# en.wikipedia.org,05-13-2020,14:38:49,0,10.0.2.15,-1,-1

def col_selector(fileType):
    switch = {
        "ping": ping_cols,
        "tr1": tr1_cols,
        "tr2": tr2_cols,
        "api": api_cols,
        "pchar_agg": pchar_agg_cols,
        "pchar1": pchar1_cols,
        "pchar2": pchar2_cols
    }
    
    return switch.get(fileType, "ERROR FILETYPE NOT FOUND: " + fileType)


In [36]:
data = {}
teams = os.listdir(datapath)
for team in teams:
    data[team] = {}
    for seq in ['seq1', 'seq2', 'seq3']:
        data[team][seq] = {}
        seq_path = datapath + '/' + team + '/clean_data/' + seq
        for file in os.listdir(seq_path):
            match = re.search('(.*)_(ping|tr1|tr2|pchar1|pchar2|pchar_agg|api)', file)
            website = match.group(1)
            fileType = match.group(2)
            if not website in data[team][seq].keys(): 
                data[team][seq][website] = {}
                
            cols = col_selector(fileType)
            df = pd.read_csv(seq_path + '/' + file, names = cols, header = 0)
            data[team][seq][website][fileType] = df

# All data is now read, can be indexed via data[team][seq#][website][fileType]
# team: Team1...Team8
# seq#: seq1, seq2, seq3
# website: e.g. www_cnn_com
# fileType: one of ping|tr1|tr2|pchar1|pchar2|pchar_agg|api